In [1]:
import sys, os
sys.path.append(os.path.abspath('../'))

In [2]:
from src.utils.flow import *
from src.utils.flow_calculations import *
from src.utils.restoration import *
from src.utils.truncated_packet import *
from src.operations.size_perturbation_logic import *
from src.operations.timing_perturbation_logic import *
from src.operations.calculate_fitness import *

modified_pcap_path = "../data/interim/testing_small_perturbed.pcapng"
pcap_file_path = '../data/raw/botnet-capture-20110816-donbot.pcap'

In [3]:
truncated_packets = create_truncated_packets_from_pcap(pcap_file_path)
truncated_packets = assign_flow_ids_to_packets(truncated_packets)
flow_id = 1

In [4]:
import pygad
import numpy as np
            


In [13]:
import pygad
import copy

class FlowOptimizer:
    def __init__(self, packets, model_name, flow_id):
        self.packets = packets
        self.model_name = model_name
        self.flow_id = flow_id
        self.ga_instance = None
        
    def apply_best_solution(self):
        best_solution, best_solution_fitness, _ = self.ga_instance.best_solution()
        best_operations = self.decode_operations(best_solution)
        # Stosujemy najlepsze operacje do pakietów
        self.best_modified_packets = self.apply_operations_to_packets(best_operations, self.packets, self.flow_id)
        # Możesz zwrócić więcej informacji o najlepszym rozwiązaniu, jeśli potrzebujesz
        return self.best_modified_packets, best_solution_fitness

    def on_generation(self, ga_instance):
        current_population = ga_instance.population
        print(f"Flow ID: {self.flow_id}, Pokolenie: {ga_instance.generations_completed}")
        #print("Aktualna populacja:")
        #for solution in current_population:
        #    print(solution)
        # Tworzymy wiadomość zawierającą interesujące nas informacje
        message = f"Najlepszy wynik: {ga_instance.best_solution()[1]}\n"
        print(message)  # Wyświetlamy informacje na konsoli
        
        # Zapisujemy te same informacje do pliku
        #with open(f"progress_for_flow_{self.flow_id}.txt", "a") as file:
        #    file.write(message)
            
    def on_fitness(self, ga_instance):
        ga_instance.plot_fitness()

    def fitness_function(self, ga_instance, solution, solution_idx):
        operations = self.decode_operations(solution)
        modified_packets = self.apply_operations_to_packets(operations)
        
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitness = 1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats)
        
        return fitness

    def optimize_for_flow(self):
        
        #with open(f"progress_for_flow_{self.flow_id}.txt", "w") as file:
        #    file.write("")  # Tylko czyszczenie zawartości pliku
            
        ga_instance = pygad.GA(num_generations=250,
                               num_parents_mating=1,
                               fitness_func=self.fitness_function,
                               sol_per_pop=5,
                               #initial_population=np.array([
                               #                                 [1, 0.1, 1, 0],  # Rozwiązanie 1
                               #                                 [2, 0.3, 0, 2], # Rozwiązanie 2
                               #                             ]),
                               on_generation=self.on_generation,
                               num_genes=4,  # Przykład, dostosuj do ilości operacji
                               gene_space = [[0, 1, 2, 3],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2]]) # focus point 

        
        ga_instance.run()
        print(ga_instance.best_solution())

    def decode_operations(self, genotype):
        operations = []
        operation_id = genotype[0]  # Typ operacji

        # Dekodowanie i skalowanie scaling_factor
        scaling_factor_normalized = genotype[1]
        if operation_id == 0:  # size uniform
            scaling_factor = 0.25 + (scaling_factor_normalized * (10.0 - 0.25))
        elif operation_id == 1:  # size normal
            scaling_factor = 1.1 + (scaling_factor_normalized * (5.0 - 1.1))
        elif operation_id == 2:  # time uniform
            scaling_factor = 0.4 + (scaling_factor_normalized * (2.4 - 0.4))
        elif operation_id == 3:  # time normal
            scaling_factor = 1.1 + (scaling_factor_normalized * (3.0 - 1.1))

        # Interpretacja direction (nie wymaga dodatkowej logiki, przy założeniu że jest to wartość dyskretna 0, 1, 2)
        direction = None
        if operation_id in [0, 1]:  # Jeśli operacja dotyczy rozmiaru
            direction = genotype[2]  

        # Interpretacja focus_point jako ciąg znaków
        focus_point_map = {0: "start", 1: "middle", 2: "end"}
        focus_point = None
        if operation_id in [1, 3]:  # Jeśli operacja to size normal lub time normal
            focus_point_index = genotype[3]  # Pobieramy indeks
            focus_point = focus_point_map.get(focus_point_index, None)  # Mapujemy indeks na ciąg znaków

        operation = {
            'operation_id': operation_id,
            'scaling_factor': scaling_factor,
            'direction': direction,
            'focus_point': focus_point
        }
        operations.append(operation)
        return operations


    def apply_operations_to_packets(self, operations):
        for operation in operations:
            operation_id = operation['operation_id']
            scaling_factor = operation['scaling_factor']
            direction = operation.get('direction')
            focus_point = operation.get('focus_point')

            if operation_id == 0:  # size uniform
                # focus point: irrelevent
                # direction: [0, 1, 2]
                
                packets = adjust_packet_size(self.packets, self.flow_id, direction, 'uniform', scaling_factor)
                
            elif operation_id == 1:  # size normal
                # if focus 
                packets = adjust_packet_size(self.packets, self.flow_id, direction, 'normal', scaling_factor, focus_point)
                
            elif operation_id == 2:  # time uniform
                packets = apply_perturbation(self.packets, self.flow_id, 'uniform', scaling_factor)
                
            elif operation_id == 3:  # time normal
                # if focus
                packets = apply_time_perturbation_with_focus(self.packets, self.flow_id, 'normal', focus_point, scaling_factor)
                
        return packets
    





model_name = 'botnet-capture-20110815-fast-flux-2'
optimizers = []  # Lista na instancje optymalizatorów

flow_ids = [28, 61, 117]  # Przykładowe ID przepływów do optymalizacji

for flow_id in flow_ids:
    optimizer = FlowOptimizer(truncated_packets, model_name, flow_id)
    optimizer.optimize_for_flow()
    optimizers.append(optimizer)
    #print(optimizer.best_solution())
    


/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:728: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Flow ID: 28, Pokolenie: 1
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 2
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 3
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 4
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 5
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 6
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 7
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 8
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 9
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 10
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 11
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 12
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 13
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 14
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 15
Najlepszy wynik: 0.11970576275351608

Flow ID: 28, Pokolenie: 16
Najleps

/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:728: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 2
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 3
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 4
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 5
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 6
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 7
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 8
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 9
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 10
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 11
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 12
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 13
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 14
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 15
Najlepszy wynik: 0.12993872344464252

Flow ID: 61, Pokolenie: 16
Najlepszy wynik: 0.12993872344464

/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:728: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Flow ID: 117, Pokolenie: 1
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 2
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 3
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 4
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 5
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 6
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 7
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 8
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 9
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 10
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 11
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 12
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 13
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 14
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokolenie: 15
Najlepszy wynik: 0.16567391737523196

Flow ID: 117, Pokol

In [6]:
# SEQUENTIAL:

In [14]:
import pygad
import copy

class FlowOptimizer:
    def __init__(self, packets, model_name, flow_id):
        self.packets = packets
        self.model_name = model_name
        self.flow_id = flow_id
        self.ga_instance = None
        
    def apply_best_solution(self):
        best_solution, best_solution_fitness, _ = self.ga_instance.best_solution()
        best_operations = self.decode_operations(best_solution)
        # Stosujemy najlepsze operacje do pakietów
        self.best_modified_packets = self.apply_operations_to_packets(best_operations, self.packets, self.flow_id)
        # Możesz zwrócić więcej informacji o najlepszym rozwiązaniu, jeśli potrzebujesz
        return self.best_modified_packets, best_solution_fitness

    def on_generation(self, ga_instance):
        current_population = ga_instance.population
        print(f"Flow ID: {self.flow_id}, Pokolenie: {ga_instance.generations_completed}")
        #print("Aktualna populacja:")
        #for solution in current_population:
        #    print(solution)
        # Tworzymy wiadomość zawierającą interesujące nas informacje
        message = f"Najlepszy wynik: {ga_instance.best_solution()[1]}\n"
        print(message)  # Wyświetlamy informacje na konsoli
        
        # Zapisujemy te same informacje do pliku
        #with open(f"progress_for_flow_{self.flow_id}.txt", "a") as file:
        #    file.write(message)
            
    def on_fitness(self, ga_instance):
        ga_instance.plot_fitness()

    def fitness_function(self, ga_instance, solution, solution_idx):
        operations = self.decode_operations(solution)
        modified_packets = self.apply_operations_to_packets(operations)
        
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
            
        fitness = 1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats)
        
        return fitness

    def optimize_for_flow(self):
        
        #with open(f"progress_for_flow_{self.flow_id}.txt", "w") as file:
        #    file.write("")  # Tylko czyszczenie zawartości pliku
            
        ga_instance = pygad.GA(num_generations=500,
                               num_parents_mating=1,
                               fitness_func=self.fitness_function,
                               sol_per_pop=3,
                               mutation_type = 'random',
                               mutation_probability = 0.45,
                               
                               initial_population=np.array([
                                                                [0, 0.07, 1, 0, 1, 0.3, 1, 0, 2, 0.4, 0, 1, 3, 0.01, 0, 2, 2, 0.3, 1, 1, 4, 0.002, 0, 1, 4, 0.1, 1, 1, 2, 0.3, 1, 1, 3, 0.01, 0, 2, 1, 0.7, 2, 2],  # Rozwiązanie 1
                                                                [1, 0.3, 1, 0, 0, 0.6, 0, 0, 2, 0.4, 0, 1, 3, 0.01, 0, 2, 4, 0.002, 0, 1, 1, 0.7, 2, 2, 0, 0.6, 0, 0, 4, 0.006, 0, 1, 2, 0.3, 1, 1, 2, 0.45, 0, 1], # Rozwiązanie 2
                                                                [2, 0.4, 0, 1, 3, 0.5, 0, 0, 0, 0.6, 0, 0, 4, 0.002, 0, 1, 3, 0.5, 0, 0, 2, 0.3, 1, 1, 0, 0.61, 0, 0, 0, 0.07, 1, 0, 1, 0.3, 1, 0, 0, 0.6, 0, 0],
                                                                
                                                           ]),
                               on_generation=self.on_generation,
                               num_genes=40,  # Przykład, dostosuj do ilości operacji
                               gene_space = [[0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2],
                                            [0, 1, 2, 3, 4],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2]]) # focus point 

        
        ga_instance.run()
        print(ga_instance.best_solution())

    def decode_operations(self, genotype):
        operations = []
        print(len(genotype))
        for i in range(0, len(genotype), 4):  # Iteracja przez genotyp co 4 geny (jedna operacja)
            if genotype[i] == 4:  # Sprawdzenie, czy operacja jest "pusta"
                continue  # Pomijamy "pustą" operację
            
            operation_id = genotype[i]  # Typ operacji

            # Dekodowanie i skalowanie scaling_factor
            scaling_factor_normalized = genotype[i + 1]
            if operation_id == 0:  # size uniform
                scaling_factor = 0.25 + (scaling_factor_normalized * (10.0 - 0.25))
            elif operation_id == 1:  # size normal
                scaling_factor = 1.1 + (scaling_factor_normalized * (5.0 - 1.1))
            elif operation_id == 2:  # time uniform
                scaling_factor = 0.4 + (scaling_factor_normalized * (2.4 - 0.4))
            elif operation_id == 3:  # time normal
                scaling_factor = 1.1 + (scaling_factor_normalized * (3.0 - 1.1))

            # Interpretacja direction (nie wymaga dodatkowej logiki, przy założeniu że jest to wartość dyskretna 0, 1, 2)
            direction = None
            if operation_id in [0, 1]:  # Jeśli operacja dotyczy rozmiaru
                direction = genotype[i+ 2]  

            # Interpretacja focus_point jako ciąg znaków
            focus_point_map = {0: "start", 1: "middle", 2: "end"}
            focus_point = None
            if operation_id in [1, 3]:  # Jeśli operacja to size normal lub time normal
                focus_point_index = genotype[i+3]  # Pobieramy indeks
                focus_point = focus_point_map.get(focus_point_index, None)  # Mapujemy indeks na ciąg znaków

            operation = {
                'operation_id': operation_id,
                'scaling_factor': scaling_factor,
                'direction': direction,
                'focus_point': focus_point
            }
            operations.append(operation)
        #print(operations)
        return operations


    def apply_operations_to_packets(self, operations):
        for operation in operations:
            operation_id = operation['operation_id']
            scaling_factor = operation['scaling_factor']
            direction = operation.get('direction')
            focus_point = operation.get('focus_point')
            
            

            if operation_id == 0:  # size uniform
                # focus point: irrelevent
                # direction: [0, 1, 2]
                
                packets = adjust_packet_size(self.packets, self.flow_id, direction, 'uniform', scaling_factor)
                
            elif operation_id == 1:  # size normal
                # if focus 
                packets = adjust_packet_size(self.packets, self.flow_id, direction, 'normal', scaling_factor, focus_point)
                
            elif operation_id == 2:  # time uniform
                packets = apply_perturbation(self.packets, self.flow_id, 'uniform', scaling_factor)
                
            elif operation_id == 3:  # time normal
                # if focus
                packets = apply_time_perturbation_with_focus(self.packets, self.flow_id, 'normal', focus_point, scaling_factor)
                
        return packets
    





model_name = 'botnet-capture-20110815-fast-flux-2'
optimizers = []  # Lista na instancje optymalizatorów

flow_ids = [28, 61, 117]  # Przykładowe ID przepływów do optymalizacji

for flow_id in flow_ids:
    optimizer = FlowOptimizer(truncated_packets, model_name, flow_id)
    optimizer.optimize_for_flow()
    optimizers.append(optimizer)
    #print(optimizer.best_solution())
    


/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


40
40
40
40
40
Flow ID: 28, Pokolenie: 1
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 2
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 3
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 4
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 5
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 6
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 7
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 8
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 9
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 10
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 11
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 12
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokolenie: 13
40
40
Najlepszy wynik: 0.160523653054223

40
40
Flow ID: 28, Pokole

/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 3
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 4
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 5
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 6
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 7
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 8
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 9
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 10
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 11
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 12
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 13
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 14
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 61, Pokolenie: 15
40
40
Najlep

/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


40
40
Flow ID: 117, Pokolenie: 1
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 2
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 3
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 4
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 5
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 6
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 7
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 8
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 9
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 10
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 11
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 12
40
40
Najlepszy wynik: 0.16567391737523196

40
40
Flow ID: 117, Pokolenie: 13
40
40
Najlepszy wynik: 0.16567391737523

In [ ]:
# MULTIPLE SIGNLE RUN:

In [4]:
best_solution[1]
# TODO: odtworzyc to na truncated packet

NameError: name 'best_solution' is not defined

In [48]:
import pygad
import copy

class FlowOptimizer:
    def __init__(self, packets, model_name, flow_id):
        self.packets = packets
        self.model_name = model_name
        self.flow_id = flow_id
        self.ga_instance = None
        
    def apply_best_solution(self):
        best_solution, best_solution_fitness, _ = self.ga_instance.best_solution()
        best_operations = self.decode_operations(best_solution)
        # Stosujemy najlepsze operacje do pakietów
        self.best_modified_packets = self.apply_operations_to_packets(best_operations, self.packets, self.flow_id)
        # Możesz zwrócić więcej informacji o najlepszym rozwiązaniu, jeśli potrzebujesz
        return self.best_modified_packets, best_solution_fitness

    def on_generation(self, ga_instance):
        current_population = ga_instance.population
        print(f"Flow ID: {self.flow_id}, Pokolenie: {ga_instance.generations_completed}")
        #print("Aktualna populacja:")
        #for solution in current_population:
        #    print(solution)
        # Tworzymy wiadomość zawierającą interesujące nas informacje
        message = f"Najlepszy wynik: {ga_instance.best_solution()[1]}\n"
        print(message)  # Wyświetlamy informacje na konsoli
        
        # Zapisujemy te same informacje do pliku
        #with open(f"progress_for_flow_{self.flow_id}.txt", "a") as file:
        #    file.write(message)
            
    def on_fitness(self, ga_instance):
        ga_instance.plot_fitness()

    def fitness_function(self, ga_instance, solution, solution_idx):
        operations = self.decode_operations(solution)
        modified_packets = self.apply_operations_to_packets(operations)
        
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitness = 1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats)
        
        return fitness

    def optimize_for_flow(self):
        
        #with open(f"progress_for_flow_{self.flow_id}.txt", "w") as file:
        #    file.write("")  # Tylko czyszczenie zawartości pliku
            
        ga_instance = pygad.GA(num_generations=25,
                               num_parents_mating=3,
                               fitness_func=self.fitness_function,
                               sol_per_pop=13,
                               initial_population=np.array([
                                                                [0, 0.7, 1, 0],  # size uni +
                                                                [0, 0.3, 0, 2],  # size uni -
                                                                [2, 0.75, 0, 2], # time uni +
                                                                [2, 0.25, 1, 0], # time uni - 
                                                                [4, 0.5, 0, 0], # no operation
                                                                [1, 0.2, 1, 0], # size low bi
                                                                [1, 0.24, 0, 1], # size mid bi
                                                                [1, 0.54, 1, 1], # size mid fwd
                                                                [1, 0.7, 2, 1], # size mid bwd
                                                                [1, 0.61, 0, 2], # size high bi
                                                                [3, 0.8, 0, 0], # time low
                                                                [3, 0.26, 1, 1], # time mid
                                                                [3, 0.4, 0, 2], # time high
                                                            ]),
                               on_generation=self.on_generation,
                               num_genes=4,  # Przykład, dostosuj do ilości operacji
                               gene_space = [[0, 1, 2, 3],  # op.type: 0-size uniform, 1-size normal, 2-time uniform, 3-time normal 
                                            {'low': 0.0, 'high': 1.0},  # scaling_factor - zakres od 0 do 1, będzie skalowany odpowiednio do operacji
                                            [0,1,2],  # direction
                                            [0,1,2]]) # focus point 

        
        ga_instance.run()
        print(ga_instance.best_solution())

    def decode_operations(self, genotype):
        #operations = []
        operation_id = genotype[0]  # Typ operacji

        # Dekodowanie i skalowanie scaling_factor
        scaling_factor_normalized = genotype[1]
        if operation_id == 0:  # size uniform
            scaling_factor = 0.25 + (scaling_factor_normalized * (10.0 - 0.25))
        elif operation_id == 1:  # size normal
            scaling_factor = 1.1 + (scaling_factor_normalized * (5.0 - 1.1))
        elif operation_id == 2:  # time uniform
            scaling_factor = 0.4 + (scaling_factor_normalized * (2.4 - 0.4))
        elif operation_id == 3:  # time normal
            scaling_factor = 1.1 + (scaling_factor_normalized * (3.0 - 1.1))
        elif operation_id == 4:
            direction = None
            focus_point = None
            scaling_factor = None

        # Interpretacja direction (nie wymaga dodatkowej logiki, przy założeniu że jest to wartość dyskretna 0, 1, 2)
        direction = None
        if operation_id in [0, 1]:  # Jeśli operacja dotyczy rozmiaru
            direction = genotype[2]  

        # Interpretacja focus_point jako ciąg znaków
        focus_point_map = {0: "start", 1: "middle", 2: "end"}
        focus_point = None
        if operation_id in [1, 3]:  # Jeśli operacja to size normal lub time normal
            focus_point_index = genotype[3]  # Pobieramy indeks
            focus_point = focus_point_map.get(focus_point_index, None)  # Mapujemy indeks na ciąg znaków

        operation = {
            'operation_id': operation_id,
            'scaling_factor': scaling_factor,
            'direction': direction,
            'focus_point': focus_point
        }
        #operations.append(operation)
        return operation


    def apply_operations_to_packets(self, operation):
        #for operation in operations:
        operation_id = operation['operation_id']
        if operation_id == 4:
            return self.packets
        scaling_factor = operation['scaling_factor']
        direction = operation.get('direction')
        focus_point = operation.get('focus_point')

        if operation_id == 0:  # size uniform
                # focus point: irrelevent
                # direction: [0, 1, 2]
                
            packets = adjust_packet_size(self.packets, self.flow_id, direction, 'uniform', scaling_factor)
                
        elif operation_id == 1:  # size normal
                # if focus 
            packets = adjust_packet_size(self.packets, self.flow_id, direction, 'normal', scaling_factor, focus_point)
                
        elif operation_id == 2:  # time uniform
            packets = apply_perturbation(self.packets, self.flow_id, 'uniform', scaling_factor)
                
        elif operation_id == 3:  # time normal
                # if focus
            packets = apply_time_perturbation_with_focus(self.packets, self.flow_id, 'normal', focus_point, scaling_factor)
                
        return packets
    





model_name = 'botnet-capture-20110815-fast-flux-2'
optimizers = []  # Lista na instancje optymalizatorów

flow_ids = [2, 61, 117]  # Przykładowe ID przepływów do optymalizacji

for flow_id in flow_ids:
    optimizer = FlowOptimizer(truncated_packets, model_name, flow_id)
    optimizer.optimize_for_flow()
    optimizers.append(optimizer)
    #print(optimizer.best_solution())
    


/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:728: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Flow ID: 2, Pokolenie: 1
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 2
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 3
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 4
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 5
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 6
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 7
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 8
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 9
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 10
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 11
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 12
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 13
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 14
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 15
Najlepszy wynik: 0.11970101666903576

Flow ID: 2, Pokolenie: 16
Najlepszy wynik: 0.1197

/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:728: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Flow ID: 61, Pokolenie: 1
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 2
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 3
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 4
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 5
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 6
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 7
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 8
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 9
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 10
Najlepszy wynik: 0.12094727586329501

Flow ID: 61, Pokolenie: 11
Najlepszy wynik: 0.12831791788854063

Flow ID: 61, Pokolenie: 12
Najlepszy wynik: 0.12831791788854063

Flow ID: 61, Pokolenie: 13
Najlepszy wynik: 0.13663072155210443

Flow ID: 61, Pokolenie: 14
Najlepszy wynik: 0.13663072155210443

Flow ID: 61, Pokolenie: 15
Najlepszy wynik: 0.13663072155210443

Flow ID: 61, Pokolenie: 16
Najleps

/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:728: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Flow ID: 117, Pokolenie: 1
Najlepszy wynik: 0.12993872344464252

Flow ID: 117, Pokolenie: 2
Najlepszy wynik: 0.12993872344464252

Flow ID: 117, Pokolenie: 3
Najlepszy wynik: 0.12993872344464252

Flow ID: 117, Pokolenie: 4
Najlepszy wynik: 0.13671755479683245

Flow ID: 117, Pokolenie: 5
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 6
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 7
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 8
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 9
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 10
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 11
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 12
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 13
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 14
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokolenie: 15
Najlepszy wynik: 0.13663072155210443

Flow ID: 117, Pokol

In [ ]:
# dla kazdego wszystkie operacje tylko losowac im atrybuty koncepcja

In [199]:
import pygad
import copy

class FlowOptimizer:
    def __init__(self, packets, model_name, flow_id, max_fitness):
        self.packets = packets
        self.model_name = model_name
        self.flow_id = flow_id
        self.ga_instance = None
        self.max_fitness = [0, 0, 0]
        
    def apply_best_solution(self):
        best_solution, best_solution_fitness, _ = self.ga_instance.best_solution()
        best_operations = self.decode_operations(best_solution)
        # Stosujemy najlepsze operacje do pakietów
        self.best_modified_packets = self.apply_operations_to_packets(best_operations, self.packets, self.flow_id)
        # Możesz zwrócić więcej informacji o najlepszym rozwiązaniu, jeśli potrzebujesz
        return self.best_modified_packets, best_solution_fitness

    def on_generation(self, ga_instance):
        current_population = ga_instance.population
        print(f"Flow ID: {self.flow_id}, Pokolenie: {ga_instance.generations_completed}")
        #print("Aktualna populacja:")
        #for solution in current_population:
        #    print(solution)
        # Tworzymy wiadomość zawierającą interesujące nas informacje
        message = f"Najlepszy wynik: {ga_instance.best_solution()[1]}\n"
        print(message)  # Wyświetlamy informacje na konsoli
        
        # Zapisujemy te same informacje do pliku
        #with open(f"progress_for_flow_{self.flow_id}.txt", "a") as file:
        #    file.write(message)
            
    def on_fitness(self, ga_instance):
        ga_instance.plot_fitness()

    def fitness_function(self, ga_instance, solution, solution_idx):
        
        fitnesses = []
        
        operations = self.decode_operations(solution)
        
        modified_packets = self.packets # always check nop
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[0], op_type = 'size_norm')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[1], op_type = 'size_uni')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[2], op_type = 'time_norm')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        
        modified_packets = self.apply_operations_to_packets(operations[3], op_type = 'time_uni')
        sizing_stats = prepare_size_stats(modified_packets, self.flow_id)
        timing_stats = prepare_timing_stats(modified_packets, self.flow_id)
        fitnesses.append(1.0 - predict_single_flow(self.model_name, sizing_stats, timing_stats))
        
        #print(fitnesses)
        max_fitness_value = max(fitnesses)
        max_fitness_index = fitnesses.index(max_fitness_value)
        print(max_fitness_index)

        # Zapisz maksymalną wartość fitness i indeks, jeśli jest większa niż obecnie zapisana maksymalna wartość
        if max_fitness_value > self.max_fitness[0]:
            self.max_fitness = [max_fitness_value, solution, max_fitness_index]
            #print(f'self:{self.max_fitness[0]}')
        
        #print(max_fitness_value)
        #print(f'solution:{solution}')
        
        return max_fitness_value

    def optimize_for_flow(self):
        
        #with open(f"progress_for_flow_{self.flow_id}.txt", "w") as file:
        #    file.write("")  # Tylko czyszczenie zawartości pliku
            
        ga_instance = pygad.GA(num_generations=4,
                               num_parents_mating=3,
                               fitness_func=self.fitness_function,
                               sol_per_pop=5,
                               parallel_processing=4,                            
                               mutation_probability=0.7,
                               initial_population=np.array([
            [5.0, 5.0, 3.0, 2.4, 2, 2],
            [2.71170068, 7.79033802, 1.79336516, 2.15132301, 2, 0],
            [3.61738866, 1.23204186, 1.11382619, 2.05771883, 0, 0],
            [4.65393677, 1.28284094, 2.82073356, 0.66388122, 2, 1],
            [1.37292894, 6.82353545, 2.06952233, 0.59225478, 2, 0],
            [2.53531317, 6.24282794, 1.24700844, 2.13030817, 1, 1],
            [3.85830277, 4.87223136, 2.0923298, 1.44531362, 1, 0],
            [4.14645349, 9.76164109, 1.67182642, 2.09448623, 1, 0],
            [2.80090477, 4.24612439, 1.41426283, 1.12520895, 2, 1],
            [1.31962389, 1.47082939, 2.15440462, 1.3226799, 2, 1]
        ]),
                               on_generation=self.on_generation,
                               num_genes=6,  # Przykład, dostosuj do ilości operacji
                               gene_space = [{'low': 1.1, 'high': 5.0},  # size normal
                                             {'low': 0.25, 'high': 5.0}, # packet size uniform
                                             {'low': 1.1, 'high': 3.0}, # time normal
                                             {'low': 0.4, 'high': 2.4}, # time uniform
                                            [0,1,2],  # direction
                                            [0,1,2]]) # focus point 

        
        ga_instance.run()
        return ga_instance, self.max_fitness

    def decode_operations(self, genotype):
        operations = []
        

        focus_point_map = {0: "start", 1: "middle", 2: "end"}
        focus_point = None
        focus_point_index = genotype[5] 
        focus_point = focus_point_map.get(focus_point_index, None)  

        size_norm = {
            'scaling_factor': genotype[0],
            'direction': genotype[4],
            'focus_point': focus_point
        }
        
        size_uni = {
            'scaling_factor': genotype[1],
            'direction': genotype[4],
            'focus_point': focus_point
        }
        
        time_norm = {
            'scaling_factor': genotype[2],
            'direction': genotype[4],
            'focus_point': focus_point
        }
        
        
        time_uni = {
            'scaling_factor': genotype[3],
            'direction': genotype[4],
            'focus_point': focus_point
        }  
            
            
        
        operations = [size_norm, size_uni, time_norm, time_uni]
        return operations 
        


    def apply_operations_to_packets(self, operation, op_type):

        scaling_factor = operation['scaling_factor']
        direction = operation.get('direction')
        focus_point = operation.get('focus_point')

        if op_type == 'size_norm':
            packets = adjust_packet_size(self.packets, self.flow_id, direction, 'normal', scaling_factor, focus_point)
            
        elif op_type == 'size_uni':
            packets = adjust_packet_size(self.packets, self.flow_id, direction, 'uniform', scaling_factor)
            
        elif op_type == 'time_norm':
            packets = apply_time_perturbation_with_focus(self.packets, self.flow_id, 'normal', focus_point, scaling_factor)
            
        elif op_type == 'time_uni':
            packets = apply_uniform_perturbation(self.packets, self.flow_id, scaling_factor)
        else:
            packets = self.packets
            
        return packets
    





model_name = 'botnet-capture-20110815-fast-flux-2'
optimizers = []  # Lista na instancje optymalizatorów

flow_ids = [69]  # Przykładowe ID przepływów do optymalizacji

for flow_id in flow_ids:
    max_fitness = [0,0,0]
    previous_max = 0
    for i in range(0, 5):
        timing_stats = prepare_timing_stats(truncated_packets, flow_id)
        sizing_stats = prepare_size_stats(truncated_packets,flow_id)
        print(predict_single_flow(model_name, sizing_stats, timing_stats))
        optimizer = FlowOptimizer(copy.deepcopy(truncated_packets), model_name, flow_id, max_fitness)

        
        max_fitness = optimizer.optimize_for_flow()
        
        print(f'nowy: {max_fitness[1][0]} stary: {previous_max}')
        if max_fitness[1][0] > previous_max: # if highest value higher than before
            
            truncated_packets = apply_best_on_packets(truncated_packets, max_fitness, flow_id)
            previous_max = max_fitness[1][0]
            
        

    #print(optimizer.best_solution())
    


0.879052724136705


/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Flow ID: 69, Pokolenie: 1
0
0
0
0
0
0
0
0
0
Najlepszy wynik: 0.12094727586329501

0
0
0
0
0
4
4
4
0
Flow ID: 69, Pokolenie: 2
0
0
0
0
0
4
0
4
0
Najlepszy wynik: 0.12094727586329501

0
0
0
0
0
4
4
4
1
Flow ID: 69, Pokolenie: 3
4
0
0
0
4
4
3
3
0
Najlepszy wynik: 0.12094727586329501

0
0
0
0
0
0
0
0
0
Flow ID: 69, Pokolenie: 4
44

4
3
00
0

0
3
Najlepszy wynik: 0.12094727586329501

nowy: 0.12094727586329501 stary: 0
0
0.879052724136705


/Users/ppopiolek/miniconda3/envs/cyberaml/lib/python3.11/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


0
0
0
0
00

0
0
0
0
0
0
0
0
0
0
0
0
0
Flow ID: 69, Pokolenie: 1
0
0
0
0


KeyboardInterrupt: 

In [189]:
def apply_best_on_packets(packets, max_fitness, flow_id):

    index = max_fitness[1][2]
    print(index)
    if index == 0:
        return packets
    
    params = optimizer.decode_operations(max_fitness[1][1])[index] # best technique
    print(f'saved:{params}')
    
    if index == 1:
        packets = adjust_packet_size(packets, flow_id, params['direction'], 'normal', params['scaling_factor'], params['focus_point'])
                
    elif index == 2:
        packets = adjust_packet_size(packets, flow_id, params['direction'], 'uniform', params['scaling_factor'])
                
    elif index == 3:
        packets = apply_time_perturbation_with_focus(packets, flow_id, 'normal', params['focus_point'], params['scaling_factor'])
                
    elif index == 4:
        packets = apply_uniform_perturbation(packets, flow_id, params['scaling_factor'])
        
    return packets

In [185]:
max_fitness[1]

[0.16567391737523196,
 array([1.57195052, 0.94143411, 2.15440462, 1.48045635, 1.        ,
        2.        ]),
 1]

In [150]:
truncated_packets

[TruncatedPacket(packet_id=1, timestamp=1313482194.088306, size=1500, pseudo_hash='1838186627723694656', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=1, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=0, psh=0, urg=0),
 TruncatedPacket(packet_id=2, timestamp=1313482194.088306000000000000, size=1500, pseudo_hash='1838186627723694656', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=1, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=0, psh=0, urg=0),
 TruncatedPacket(packet_id=3, timestamp=1313482194.088306000000000000, size=1500, pseudo_hash='1838186627723694656', flow_id=1, direction='2', src_ip=91.212.135.158, fragmented=1, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=1, psh=0, urg=0),
 TruncatedPacket(packet_id=4, timestamp=1313482194.088306000000000000, size=1500, pseudo_hash='1838186627723694656', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=1, tcp=1, udp=0, fin=0, syn=0, rst=0, ack=1, psh=0, urg=0),
 TruncatedPacket(packet_id=5, timestamp=1313482194.0883060000000000

In [94]:
def count_directions(truncated_packets, flow_id):
    direction_counts = {"1": 0, "2": 0}
    for packet in truncated_packets:
        if packet.flow_id == flow_id:
            # Ensure the direction is treated as a string
            direction_str = str(packet.direction)
            direction_counts[direction_str] += 1
    return direction_counts

# Example usage
for i in range (1, 150):
    direction_counts = count_directions(truncated_packets, i)
    print(f'{i} {direction_counts}')


1 {'1': 1090, '2': 635}
2 {'1': 2, '2': 1}
3 {'1': 2, '2': 1}
4 {'1': 2, '2': 1}
5 {'1': 14, '2': 6}
6 {'1': 4, '2': 0}
7 {'1': 4, '2': 0}
8 {'1': 4, '2': 0}
9 {'1': 4, '2': 0}
10 {'1': 4, '2': 0}
11 {'1': 4, '2': 0}
12 {'1': 4, '2': 0}
13 {'1': 4, '2': 0}
14 {'1': 4, '2': 0}
15 {'1': 4, '2': 0}
16 {'1': 6, '2': 0}
17 {'1': 6, '2': 0}
18 {'1': 6, '2': 0}
19 {'1': 6, '2': 0}
20 {'1': 6, '2': 0}
21 {'1': 6, '2': 0}
22 {'1': 6, '2': 0}
23 {'1': 6, '2': 0}
24 {'1': 6, '2': 0}
25 {'1': 6, '2': 0}
26 {'1': 1, '2': 2}
27 {'1': 1, '2': 2}
28 {'1': 1, '2': 2}
29 {'1': 2, '2': 0}
30 {'1': 2, '2': 0}
31 {'1': 2, '2': 0}
32 {'1': 2, '2': 0}
33 {'1': 2, '2': 0}
34 {'1': 14, '2': 6}
35 {'1': 4, '2': 0}
36 {'1': 4, '2': 0}
37 {'1': 4, '2': 0}
38 {'1': 4, '2': 0}
39 {'1': 4, '2': 0}
40 {'1': 4, '2': 0}
41 {'1': 4, '2': 0}
42 {'1': 4, '2': 0}
43 {'1': 4, '2': 0}
44 {'1': 4, '2': 0}
45 {'1': 2, '2': 0}
46 {'1': 2, '2': 0}
47 {'1': 4, '2': 0}
48 {'1': 4, '2': 0}
49 {'1': 2, '2': 0}
50 {'1': 4, '2': 0}
51

In [ ]:
# funkcja odtwarzania []
# proces krokowy []
# wyfiltrowanie 100 truncated []
# skladanie 100 truncated []

In [107]:
index = max_fitness[1][2]
params = optimizer.decode_operations(max_fitness[1][1])[index] # best technique
params

{'scaling_factor': 2.1045408595258115,
 'direction': 2.0,
 'focus_point': 'start'}

In [122]:
params['scaling_factor']

2.1045408595258115

In [108]:
index # size_norm -> size_uni -> time_norm -> time_uni

3

In [109]:
calculate_delta_time_statistics(117, truncated_packets, direction=0)

{'mean': 185817792.35410616,
 'min': 185817792.35410616,
 'max': 185817792.35410616,
 'std': 0.0}

In [110]:
calculate_size_statistics(117, truncated_packets, direction=0)

{'mean': 1500.0, 'min': 1500, 'max': 1500, 'std': 0.0}

In [127]:
truncated_packets = apply_best_on_packets(truncated_packets, max_fitness, 117)

In [128]:
calculate_delta_time_statistics(117, truncated_packets, direction=0)

{'mean': 239000000.0, 'min': 239000000.0, 'max': 239000000.0, 'std': 0.0}

In [135]:
calculate_size_statistics(117, truncated_packets, direction=0)

{'mean': 750.0, 'min': 750.0, 'max': 750.0, 'std': 0.0}